In [1]:
import tensorflow as tf
import os
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import numpy as np
from nilearn import image
from nilearn import plotting
import nibabel as nib
from nilearn.plotting import plot_stat_map, plot_anat, plot_img

/localhome/mkhademi/anaconda3/envs/tf/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
data_dir = '/local-scratch/localhome/mkhademi/BOLD5000_2.0/'
run_size = 37
#dataset = tf.data.TFRecordDataset(filenames = [data_dir + 'image_data/bold5000_coco.tfrecords']).batch(run_size)
#dataset_size = 2135
dataset = tf.data.TFRecordDataset(filenames = [data_dir + 'image_data/bold5000_common.tfrecords']).batch(run_size)
dataset_size = 2864
batch_size = 10
n_epochs=50
n_it=8
def tf_parse(eg):
    example = tf.io.parse_example(
        eg, {
            'x': tf.io.FixedLenFeature(shape=(71, 89, 72, 1), dtype=tf.float32),
            'yhat': tf.io.FixedLenFeature(shape=(1000), dtype=tf.float32),
            'y_coco': tf.io.FixedLenFeature(shape=(90), dtype=tf.int64),
            'y_imagenet': tf.io.FixedLenFeature(shape=(1000), dtype=tf.int64),
            'y_super': tf.io.FixedLenFeature(shape=(12), dtype=tf.int64),
            'y_common': tf.io.FixedLenFeature(shape=(90+12), dtype=tf.int64)
        })
    return example['x'], example['y_common'] 

In [3]:
def cross_val(dataset, j, dataset_size):
    recalls = []
    aucs = []
    accs = []
    recalls_rand = []
    aucs_rand = []
    accs_rand = []
    for it in range(n_it):
        train_size_1 = np.ceil(dataset_size/run_size*it/n_it)
        test_size = np.ceil(dataset_size/run_size*1/n_it)
        train_ds_1 = dataset.take(train_size_1)
        test_ds = dataset.skip(train_size_1)
        test_ds = test_ds.take(test_size)
        train_ds_2 = dataset.skip(train_size_1 + test_size)
        train_size_2 = np.ceil(dataset_size/run_size) - train_size_1 - test_size
        train_ds_2 = train_ds_2.take(train_size_2)
        train_ds = train_ds_1.concatenate(train_ds_2)
        train_ds = train_ds.unbatch().batch(batch_size)
        train_ds = train_ds.shuffle(train_size_1 + train_size_2)
        train = train_ds.map(tf_parse)
        test = test_ds.map(tf_parse)
        train = train.map(lambda x, y: (x, tf.concat([y[:,j:j+1], tf.ones_like(y[:,j:j+1],dtype=tf.int64)-y[:,j:j+1]],axis=1))) 
        test = test.map(lambda x, y: (x, tf.concat([y[:,j:j+1], tf.ones_like(y[:,j:j+1],dtype=tf.int64)-y[:,j:j+1]],axis=1)))
        model = tf.keras.Sequential([
            tf.keras.Input(shape=(71, 89, 72, 1)),
            tf.keras.layers.Conv3D(1, 3, activation='relu', padding='same', name='conv1'),
            #tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
            tf.keras.layers.Conv3D(2, 3, activation='relu', padding='same', name='conv2'),
            tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
            tf.keras.layers.Conv3D(4, 3, padding='same', name='conv3'),
            tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(rate=0.4),
            # tf.keras.layers.Dense(100, activation='sigmoid'),
            tf.keras.layers.Dense(2),
            #tf.keras.layers.Softmax()
            ])
        model.summary()
        model.compile(
            # loss='mse',
            loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            # loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
            # optimizer=keras.optimizers.RMSprop(),
            optimizer='adam',
            # metrics=[tf.keras.metrics.MeanSquaredError()],
            metrics=[tf.keras.metrics.CategoricalAccuracy()]
        )
        model.fit(train, epochs=n_epochs)
        _, acc = model.evaluate(test)
        model_rand = tf.keras.Sequential([
            tf.keras.Input(shape=(71, 89, 72, 1)),
            tf.keras.layers.Conv3D(1, 3, activation='relu', padding='same', name='conv1'),
            #tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
            tf.keras.layers.Conv3D(2, 3, activation='relu', padding='same', name='conv2'),
            tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
            tf.keras.layers.Conv3D(4, 3, padding='same', name='conv3'),
            tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(rate=0.4),
            # tf.keras.layers.Dense(100, activation='sigmoid'),
            tf.keras.layers.Dense(2),
            #tf.keras.layers.Softmax()
            ])
        model_rand.compile(
            # loss='mse',
            loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            # loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
            # optimizer=keras.optimizers.RMSprop(),
            optimizer='adam',
            # metrics=[tf.keras.metrics.MeanSquaredError()],
            metrics=[tf.keras.metrics.CategoricalAccuracy()]
        )
        train_rand = train.map(lambda x, y: (x, tf.random.shuffle(y)))
        model_rand.fit(train_rand, epochs=n_epochs)
        _, acc_rand = model_rand.evaluate(test)
        model.add(tf.keras.layers.Softmax())
        model_rand.add(tf.keras.layers.Softmax())
        m_recall = tf.keras.metrics.Recall()
        m_auc = tf.keras.metrics.AUC()
        m_recall_rand = tf.keras.metrics.Recall()
        m_auc_rand = tf.keras.metrics.AUC()
        m_acc_rand = tf.keras.metrics.CategoricalAccuracy()
        for x_batch, y_batch in test:
            yhat = model(x_batch)
            yhat_rand = model_rand(x_batch)
            m_recall.update_state(y_batch[:,0], yhat[:,0])
            m_auc.update_state(y_batch[:,0], yhat[:,0])
            m_recall_rand.update_state(y_batch[:,0], yhat_rand[:,0])
            m_auc_rand.update_state(y_batch[:,0], yhat_rand[:,0])
            m_acc_rand.update_state(y_batch, yhat_rand)
        recalls.append(m_recall.result().numpy())
        aucs.append(m_auc.result().numpy())
        accs.append(acc)
        recalls_rand.append(m_recall_rand.result().numpy())
        aucs_rand.append(m_auc_rand.result().numpy())
        accs_rand.append(m_acc_rand.result().numpy())
        break
    return recalls, aucs, accs, recalls_rand, aucs_rand, accs_rand, model

In [4]:
def cross_val_fine(dataset, js, dataset_size, model_freeze):
    recalls = []
    aucs = []
    accs = []
    recalls_rand = []
    aucs_rand = []
    accs_rand = []
    for it in range(n_it):
        train_size_1 = np.ceil(dataset_size/run_size*it/n_it)
        test_size = np.ceil(dataset_size/run_size*1/n_it)
        train_ds_1 = dataset.take(train_size_1)
        test_ds = dataset.skip(train_size_1)
        test_ds = test_ds.take(test_size)
        train_ds_2 = dataset.skip(train_size_1 + test_size)
        train_size_2 = np.ceil(dataset_size/run_size) - train_size_1 - test_size
        train_ds_2 = train_ds_2.take(train_size_2)
        train_ds = train_ds_1.concatenate(train_ds_2)
        train_ds = train_ds.unbatch().batch(batch_size)
        train_ds = train_ds.shuffle(train_size_1 + train_size_2)
        train = train_ds.map(tf_parse)
        test = test_ds.map(tf_parse)
        train = train.map(lambda x, y: (x, tf.concat([y[:,j:j+1] for j in js], axis=1))) 
        test = test.map(lambda x, y: (x, tf.concat([y[:,j:j+1] for j in js], axis=1))) 
        model = tf.keras.Sequential()
        for layer in model_freeze.layers:
            model.add(layer)
        model.build(input_shape=(None,71, 89, 72, 1))
        model.add(tf.keras.layers.Dropout(rate=0.4))
        model.add(tf.keras.layers.Dense(len(js)))
        model.summary()
        model.compile(
            loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
            # optimizer=keras.optimizers.RMSprop(),
            optimizer='adam',
            # metrics=[tf.keras.metrics.MeanSquaredError()],
            metrics=[tf.keras.metrics.BinaryAccuracy()]
        )
        model.fit(train, epochs=n_epochs)
        _, acc = model.evaluate(test)
        model_rand = tf.keras.Sequential([
            tf.keras.Input(shape=(71, 89, 72, 1)),
            tf.keras.layers.Conv3D(1, 3, activation='relu', padding='same', name='conv1'),
            #tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
            tf.keras.layers.Conv3D(2, 3, activation='relu', padding='same', name='conv2'),
            tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
            tf.keras.layers.Conv3D(4, 3, padding='same', name='conv3'),
            tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(rate=0.4),
            # tf.keras.layers.Dense(100, activation='sigmoid'),
            tf.keras.layers.Dense(len(js)),
            #tf.keras.layers.Softmax()
            ])
        model_rand.compile(
            # loss='mse',
            # loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            loss = tf.keras.losses.BinaryCrossentropy(from_logits=False),
            # optimizer=keras.optimizers.RMSprop(),
            optimizer='adam',
            # metrics=[tf.keras.metrics.MeanSquaredError()],
            metrics=[tf.keras.metrics.BinaryAccuracy()]
        )
        train_rand = train.map(lambda x, y: (x, tf.random.shuffle(y)))
        model_rand.fit(train_rand, epochs=n_epochs)
        _, acc_rand = model_rand.evaluate(test)
        print(acc_rand)
        m_recall = tf.keras.metrics.Recall()
        m_auc = tf.keras.metrics.AUC()
        m_recall_rand = tf.keras.metrics.Recall()
        m_auc_rand = tf.keras.metrics.AUC()
        m_acc_rand = tf.keras.metrics.BinaryAccuracy()
        for x_batch, y_batch in test:
            yhat = tf.keras.activations.sigmoid(model(x_batch))
            yhat_rand = tf.keras.activations.sigmoid(model_rand(x_batch))
            m_recall.update_state(y_batch, yhat)
            m_auc.update_state(y_batch, yhat)
            m_recall_rand.update_state(y_batch, yhat_rand)
            m_auc_rand.update_state(y_batch, yhat_rand)
            m_acc_rand.update_state(y_batch, yhat_rand)
        recalls.append(m_recall.result().numpy())
        aucs.append(m_auc.result().numpy())
        accs.append(acc)
        recalls_rand.append(m_recall_rand.result().numpy())
        aucs_rand.append(m_auc_rand.result().numpy())
        accs_rand.append(m_acc_rand.result().numpy())
    return recalls, aucs, accs, recalls_rand, aucs_rand, accs_rand, model

In [5]:
j = 91  # vehicle
recalls, aucs, accs, recalls_rand, aucs_rand, accs_rand, model = cross_val(dataset, j, dataset_size)
print(recalls)
print(aucs)
print(accs)
print(recalls_rand)
print(aucs_rand)
print(accs_rand)
model.pop()
model.pop()
model.pop()
for layer in model.layers:
    layer.trainable = False
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten (Flatten)            (None, 26928)             0         
_________________________________________________________________
dropout (Dropout)            (None, 26928)             0

In [6]:
js = [j for j in range(1, 9)]
model_freeze = model
dataset_vehicle = tf.data.TFRecordDataset(filenames = [data_dir+'image_data/bold5000_common_vehicle.tfrecords']).batch(run_size)
dataset_vehicle_size = 482
recalls, aucs, accs, recalls_rand, aucs_rand, accs_rand, model_fine = cross_val_fine(dataset_vehicle, js, dataset_vehicle_size,
                                                                                     model_freeze)
print(recalls)
print(aucs)
print(accs)
print(recalls_rand)
print(aucs_rand)
print(accs_rand)
model_fine.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten (Flatten)            (None, 26928)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 26928)            

In [7]:
j = 93 # animal
recalls, aucs, accs, recalls_rand, aucs_rand, accs_rand, model = cross_val(dataset, j, dataset_size)
print(recalls)
print(aucs)
print(accs)
print(recalls_rand)
print(aucs_rand)
print(accs_rand)
model.pop()
model.pop()
model.pop()
for layer in model.layers:
    layer.trainable = False
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_20 (MaxPooling (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_21 (MaxPooling (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 26928)             0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 26928)           

In [8]:
js = [j for j in range(15, 25)]
model_freeze = model
dataset_animal = tf.data.TFRecordDataset(filenames = [data_dir + 'image_data/bold5000_common_animal.tfrecords']).batch(run_size)
dataset_animal_size = 863
recalls, aucs, accs, recalls_rand, aucs_rand, accs_rand, model_fine = cross_val_fine(dataset_animal, js,
                                                                                     dataset_animal_size, model_freeze)
print(recalls)
print(aucs)
print(accs)
print(recalls_rand)
print(aucs_rand)
print(accs_rand)
model_fine.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_20 (MaxPooling (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_21 (MaxPooling (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 26928)             0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 26928)           

In [9]:
j = 97 # food
recalls, aucs, accs, recalls_rand, aucs_rand, accs_rand, model = cross_val(dataset, j, dataset_size)
print(recalls)
print(aucs)
print(accs)
print(recalls_rand)
print(aucs_rand)
print(accs_rand)
model.pop()
model.pop()
model.pop()
for layer in model.layers:
    layer.trainable = False
model.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_40 (MaxPooling (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_41 (MaxPooling (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 26928)             0         
_________________________________________________________________
dropout_36 (Dropout)         (None, 26928)           

In [10]:
js = [j for j in range(51, 61)]
model_freeze = model
dataset_food = tf.data.TFRecordDataset(filenames = [data_dir + 'image_data/bold5000_common_food.tfrecords']).batch(run_size)
dataset_food_size = 284
recalls, aucs, accs, recalls_rand, aucs_rand, accs_rand, model_fine = cross_val_fine(dataset_food, js,
                                                                                     dataset_food_size, model_freeze)
print(recalls)
print(aucs)
print(accs)
print(recalls_rand)
print(aucs_rand)
print(accs_rand)
model_fine.summary()

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_40 (MaxPooling (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_41 (MaxPooling (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 26928)             0         
_________________________________________________________________
dropout_38 (Dropout)         (None, 26928)           

In [11]:
j = 98 # furniture
recalls, aucs, accs, recalls_rand, aucs_rand, accs_rand, model = cross_val(dataset, j, dataset_size)
print(recalls)
print(aucs)
print(accs)
print(recalls_rand)
print(aucs_rand)
print(accs_rand)
model.pop()
model.pop()
model.pop()
for layer in model.layers:
    layer.trainable = False
model.summary()

Model: "sequential_54"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_60 (MaxPooling (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_61 (MaxPooling (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_30 (Flatten)         (None, 26928)             0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 26928)           

In [12]:
js = [j for j in range(61, 65)] + [j for j in range(66, 71)]
model_freeze = model
dataset_furniture = tf.data.TFRecordDataset(filenames = [data_dir + 'image_data/bold5000_common_furniture.tfrecords']).batch(run_size)
dataset_furniture_size = 435
recalls, aucs, accs, recalls_rand, aucs_rand, accs_rand, model_fine = cross_val_fine(dataset_furniture, js,
                                                                                     dataset_furniture_size, model_freeze)
print(recalls)
print(aucs)
print(accs)
print(recalls_rand)
print(aucs_rand)
print(accs_rand)
model_fine.summary()

Model: "sequential_56"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 71, 89, 72, 1)     28        
_________________________________________________________________
conv2 (Conv3D)               (None, 71, 89, 72, 2)     56        
_________________________________________________________________
max_pooling3d_60 (MaxPooling (None, 35, 44, 36, 2)     0         
_________________________________________________________________
conv3 (Conv3D)               (None, 35, 44, 36, 4)     220       
_________________________________________________________________
max_pooling3d_61 (MaxPooling (None, 17, 22, 18, 4)     0         
_________________________________________________________________
flatten_30 (Flatten)         (None, 26928)             0         
_________________________________________________________________
dropout_56 (Dropout)         (None, 26928)           